<blockquote>
    <h1>Exercise 10.8</h1>
    <p>In Section 10.2.3, a formula for calculating PVE was given in Equation 10.8. We also saw that the PVE can be obtained using the <code>sdev</code> output of the <code>prcomp()</code> function.</p>
    <p>On the <code>USArrests</code> data, calculate PVE in two ways:</p>
    <ol>
        <li>Using the <code>sdev</code> output of the <code>prcomp()</code> function, as was done in Section 10.2.3.</li>
        <li>By applying Equation 10.8 directly. That is, use the <code>prcomp()</code> function to compute the principal component loadings. Then, use those loadings in Equation 10.8 to obtain the PVE.</li>
    </ol>
    <p>These two approaches should give the same results.</p>
    <p><i>Hint: You will only obtain the same results in 1 and 2 if the same data is used in both cases. For instance, if in 1 you performed <code>prcomp()</code> using centered and scaled variables, then you must center and scale the variables before applying Equation 10.3 in 2.</i></p>
</blockquote>

In [1]:
import pandas as pd
import numpy as np

# https://stackoverflow.com/questions/34398054/ipython-notebook-cell-multiple-outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

<h3>Exercise 10.8.1</h3>
<blockquote>
    <i></i>
</blockquote>

In [2]:
df = pd.read_csv("../../DataSets/USArrests/USArrests.csv", index_col = 0)
df.head()

,Murder,Assault,UrbanPop,Rape
Alabama,13.2,236,58,21.2
Alaska,10.0,263,48,44.5
Arizona,8.1,294,80,31.0
Arkansas,8.8,190,50,19.5
California,9.0,276,91,40.6


In [3]:
scaler = StandardScaler()
df_standardized = pd.DataFrame(
    scaler.fit_transform(df),
    index=df.index,
    columns=df.columns
)

In [4]:
n, p = df.shape
n_components = p
pipeline = Pipeline([
    ('scaling', StandardScaler()), 
    ('pca', PCA(n_components=p))
])
principal_components = pipeline.fit_transform(df)
pipeline['pca'].explained_variance_ratio_

array([0.62006039, 0.24744129, 0.0891408 , 0.04335752])

In [5]:
loadings = pipeline['pca'].components_
df_loadings = pd.DataFrame(
    pipeline['pca'].components_.T, 
    columns=[f'PC{i}' for i in range(1, n_components + 1)], 
    index=df.columns
)
for principal_component in df_loadings.columns:
    assert abs(np.sum(df_loadings[principal_component]**2) - 1) < 1e-6

In [6]:
denominator = n * np.sum(df_standardized.var(axis=None, ddof=0))
for principal_component in df_loadings.columns:
    numerator = 0
    for i in range(n):
        numerator += (df_loadings[principal_component] @ df_standardized.iloc[i])**2
        
    print(principal_component, numerator / denominator)
        

PC1 0.6200603947873735
PC2 0.2474412881349604
PC3 0.08914079514520754
PC4 0.043357521932458856
